## Baixar Arquivos

Esse notebook está responsável por baixar os arquivos do banco de dados disponível. Para isso, também, são feitas algumas análises iniciais para melhorar a compreensão dos dados disponíveis.

---

In [1]:
# importando pacotes
from dotenv import load_dotenv
import os
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# conectar com banco de dados
load_dotenv()

USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
DATABASE = os.getenv("DATABASE")
PORT = os.getenv("PORT")

conn = mysql.connector.connect(
    host = HOST,
    user = USER,
    password = PASSWORD,
    database = DATABASE,
    port = PORT
)

cursor = conn.cursor()

In [3]:
#função para auxiliar nas consultas
def executar_query(cursor, query):
    cursor.execute(query)
    
    retorno = cursor.fetchall()
    cabecalho = [col[0] for col in cursor.description]

    aux = [cabecalho] + retorno
    
    largura  = [max(len(str(linha[i])) for linha in aux) for i in range(len(cabecalho))]
     
    cabecalho = " | ".join(f"{cabecalho[i]:<{largura[i]}}" for i in range(len(cabecalho)))
    
    print(cabecalho)
    print("-" * len(cabecalho))

    for dado in retorno:
        linha = " | ".join(f"{str(dado[i]):<{largura[i]}}" for i in range(len(dado)))
        print(linha)
    print(f"{"-" * len(cabecalho)}\n\n")

In [4]:
# armazenar em uma variável todas as tabelas do banco de dados
cursor.execute("SHOW TABLES;")
tabelas = [res[0] for res in cursor.fetchall()]
print(tabelas)

['daumau', 'desinstalacoes', 'installs', 'ratings_reviews']


In [5]:
# buscar por documentação das tabelas
executar_query(cursor, f"""
SELECT TABLE_NAME, TABLE_ROWS, TABLE_COMMENT
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = '{DATABASE}'""")


TABLE_NAME      | TABLE_ROWS | TABLE_COMMENT
--------------------------------------------
daumau          | 41464      |              
desinstalacoes  | 53126      |              
installs        | 47002      |              
ratings_reviews | 45890      |              
--------------------------------------------




In [6]:
# buscar por documentação das colunas
for tabela in tabelas:
    print(f"Tabela: {tabela}")
    executar_query(cursor, f"""
SELECT COLUMN_NAME, COLUMN_COMMENT
FROM information_schema.COLUMNS
WHERE TABLE_NAME = '{tabela}'""")

Tabela: daumau
COLUMN_NAME | COLUMN_COMMENT
----------------------------
appId       |               
date        |               
dauReal     |               
mauReal     |               
----------------------------


Tabela: desinstalacoes
COLUMN_NAME    | COLUMN_COMMENT
-------------------------------
appId          |               
country        |               
date           |               
lang           |               
predictionLoss |               
-------------------------------


Tabela: installs
COLUMN_NAME | COLUMN_COMMENT
----------------------------
appid       |               
date        |               
newinstalls |               
----------------------------


Tabela: ratings_reviews
COLUMN_NAME   | COLUMN_COMMENT
------------------------------
appid         |               
category      |               
daily_ratings |               
daily_reviews |               
date          |               
ratings       |               
reviews       |               
---

In [7]:
# como não temos uma documentação das tabelas, iremos seguir com uma análise exploratória
for tabela in tabelas:
    print(f"Tabela: {tabela}\n")
    executar_query(cursor, f"SELECT * FROM {tabela} LIMIT 10")
    

Tabela: daumau

appId         | date       | dauReal   | mauReal   
---------------------------------------------------
com.app.33540 | 2024-01-01 | 393961.0  | 3643110.0 
com.app.33540 | 2024-01-02 | 800448.0  | 3643040.0 
com.app.31679 | 2024-01-01 | 157532.0  | 1349262.0 
com.app.31679 | 2024-01-02 | 313847.0  | 1349317.0 
com.app.77385 | 2024-01-01 | 239768.0  | 1934660.0 
com.app.77385 | 2024-01-02 | 465620.0  | 1936143.0 
com.app.41189 | 2024-01-01 | 4954347.0 | 19039746.0
com.app.41189 | 2024-01-02 | 8137953.0 | 19042110.0
com.app.85381 | 2024-01-01 | 1006.0    | 24799.0   
com.app.85381 | 2024-01-02 | 1624.0    | 24862.0   
---------------------------------------------------


Tabela: desinstalacoes

appId         | date                | country | lang | predictionLoss
---------------------------------------------------------------------
com.app.71740 | 2024-01-01 00:00:00 | br      | pt   | 1463          
com.app.33540 | 2024-01-01 00:00:00 | br      | pt   | 8170          
co

In [1]:
# Pelos nomes das colunas, valores, e algumas consultas pontuais, temos:
#
# daumau
#	appId - identificador do aplicativo
#	date - data (data)
#	dauReal - quantidade de usuários distintos ativos por dia
#	mauReal - quantidade de usuários distintos ativos por mês
#
# desisntalacoes
#	appId - identificador do aplicativo
#	date - data (data+hora)
#	country - país do usuário
#	lang - língua selecionada do usuário
#	predictionLoss - quantidade prevista de desinstalações do aplicativo
#
# installs
#	appid - identificador do aplicativo
#	date - data (data+hora)
#	newinstalls - quantidade de instalações do aplicativo
#
# ratings_reviews
#	date - data (data)
#	appid - identificador do aplicativo
#	category - categoria do aplicativo
#	ratings - avaliações totais até o determinado dia
#	daily_ratings - novas avaliações no dia
#	reviews - revisões totais do aplicativo
#	daily_reviews - novas revisões no dia
#
# OBSERVACOES:
#    * daumau e desisntalacoes utilizam "appId" enquanto que installs e ratings_review utilizam "appid". (Não causa erro no MySQL)
#    * daumau e ratings_review utilizam "date" apenas com data, enquanto que desisntalacoes e installs utilizam "date" com "data+hora"

In [9]:
# validar se desinstalações e instalacoes apresentam alguma variação de hora
executar_query(cursor, f"""
SELECT * from desinstalacoes
WHERE HOUR(date) + MINUTE(date) + SECOND(date) > 0
""")

executar_query(cursor, f"""
SELECT * from installs
WHERE HOUR(date) + MINUTE(date) + SECOND(date) > 0
""")

appId | date | country | lang | predictionLoss
----------------------------------------------
----------------------------------------------


appid | date | newinstalls
--------------------------
--------------------------




In [10]:
# Como não existem variações de hora, temos a confirmação que podemos cruzar os dados pelas colunas "appId" e "date" sem problemas.
# Vamos fechar a conexão atual pois não vamos explorar mais os dados por enquanto.
cursor.close()
conn.close()

In [11]:
# Pandas recomenda utilizar SQLAlchemy para armazenar os dados em um dataframe, por isso estamos abrindo novamente a conexão.
string_conector = f"mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

motor = create_engine(string_conector)

# Vamos juntar todas as tabelas e armazenar em um arquivo unificado, mandento valores nulos que vão ser tratados nos próximos passos.
query = f"""
WITH dados_unificados AS (
    	SELECT appId, date FROM daumau
    	UNION
    	SELECT appId, DATE(date) AS date from desinstalacoes
    	UNION
    	SELECT appId AS appid, DATE(date) AS date from installs
    	UNION
    	SELECT appid AS appid, date from ratings_reviews
)

SELECT
    du.appId,
    du.date,

    dau.dauReal,
    dau.mauReal,

    des.country,
    des.lang,
    des.predictionLoss,

    ins.newinstalls,

    rat.category,
    rat.ratings,
    rat.daily_ratings,
    rat.reviews,
    rat.daily_reviews
    
FROM dados_unificados AS du
    LEFT JOIN daumau AS dau
        ON dau.appId = du.appId AND dau.date = du.date
    LEFT JOIN desinstalacoes AS des
        ON des.appId = du.appId AND DATE(des.date) = du.date
    LEFT JOIN installs AS ins
        ON ins.appid = du.appId AND DATE(ins.date) = du.date
    LEFT JOIN ratings_reviews as rat
        ON rat.appid = du.appId AND rat.date = du.date
"""

df = pd.read_sql(query, motor)
motor.dispose()

In [12]:
# armazenar os dados em um arquivo
pasta_atual = os.getcwd()
pasta_pai = os.path.dirname(pasta_atual)
pasta_data = os.path.join(pasta_pai, "data")

df.to_csv(f"{pasta_data}\\dados_brutos.csv", index=False)